In [ ]:
import cv2
import mediapipe as mp
import pygame
import pyautogui
import numpy as np
import asyncio
import nest_asyncio
import time
import platform
import win32gui
import win32con

# Apply nest_asyncio for environments with existing event loops
nest_asyncio.apply()

# Initialize Pygame
pygame.init()

# Pygame window dimensions
WINDOW_WIDTH, WINDOW_HEIGHT = 640, 480
screen = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
pygame.display.set_caption("Hand Gesture Cursor Control")

# Keep window on top
hwnd = pygame.display.get_wm_info()['window']
win32gui.SetWindowPos(hwnd, win32con.HWND_TOPMOST, 0, 0, 0, 0,
                      win32con.SWP_NOMOVE | win32con.SWP_NOSIZE)

# Webcam preview dimensions
WEBCAM_WIDTH, WEBCAM_HEIGHT = 320, 240

# Colors
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
YELLOW = (255, 255, 0)
CYAN = (0, 255, 255)
ORANGE = (255, 165, 0)
MAGENTA = (255, 0, 255)

# Mirroring toggle
MIRROR_WEBCAM = True

# MediaPipe hand detection
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_drawing = mp.solutions.drawing_utils

# Webcam setup with error handling
cap = None
for index in [0, 1, 2]:
    cap = cv2.VideoCapture(index)
    if cap.isOpened():
        print(f"Webcam found at index {index}")
        break
if not cap or not cap.isOpened():
    print("Error: Could not access webcam. Check if the camera is connected and not in use.")
    exit(1)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WEBCAM_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, WEBCAM_HEIGHT)

# Get screen size for cursor mapping
SCREEN_WIDTH, SCREEN_HEIGHT = pyautogui.size()
print(f"Screen resolution: {SCREEN_WIDTH}x{SCREEN_HEIGHT}")

# PyAutoGUI settings
pyautogui.FAILSAFE = False
pyautogui.MINIMUM_DURATION = 0
pyautogui.MINIMUM_SLEEP = 0.005

# Font for instructions
font = pygame.font.SysFont("arial", 20)

# Gesture state variables
last_gesture_time = time.time()
gesture_debounce_delay = 0.5
last_cursor_x, last_cursor_y = None, None
smoothing_factor = 0.4
last_raw_x = None

def distance(point1, point2):
    """Calculate Euclidean distance between two MediaPipe landmarks."""
    return ((point1.x - point2.x) ** 2 + (point1.y - point2.y) ** 2) ** 0.5

def detect_gestures(frame):
    global last_gesture_time, last_cursor_x, last_cursor_y, MIRROR_WEBCAM, last_raw_x
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)
    
    cursor_x, cursor_y = None, None
    is_fist = False
    is_open_hand = False
    is_scroll = False
    scroll_amount = 0
    is_minimize = False
    is_close = False
    is_open_app = False
    debug_text = "No hand detected"
    raw_x = None
    direction = ""

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            # Key landmarks
            thumb_tip = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
            middle_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
            ring_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
            pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            middle_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]
            ring_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
            pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]
            thumb_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_MCP]
            
            # Raw x-coordinate
            raw_x = index_tip.x
            # Detect direction
            if last_raw_x is not None:
                if raw_x > last_raw_x + 0.01:
                    direction = "Moving right"
                elif raw_x < last_raw_x - 0.01:
                    direction = "Moving left"
            last_raw_x = raw_x
            
            # Cursor position (index finger tip)
            cursor_x = int((1 - index_tip.x if MIRROR_WEBCAM else index_tip.x) * SCREEN_WIDTH)
            cursor_y = int(index_tip.y * SCREEN_HEIGHT)
            
            # Smooth cursor
            if last_cursor_x is not None and last_cursor_y is not None:
                cursor_x = int((1 - smoothing_factor) * last_cursor_x + smoothing_factor * cursor_x)
                cursor_y = int((1 - smoothing_factor) * last_cursor_y + smoothing_factor * cursor_y)
            last_cursor_x, last_cursor_y = cursor_x, cursor_y
            
            # Distances
            thumb_index_dist = distance(thumb_tip, index_tip)
            index_middle_dist = distance(index_tip, middle_tip)
            index_to_wrist = distance(index_tip, wrist)
            middle_to_wrist = distance(middle_tip, wrist)
            ring_to_wrist = distance(ring_tip, wrist)
            pinky_to_wrist = distance(pinky_tip, wrist)
            thumb_to_wrist = distance(thumb_tip, wrist)
            index_to_mcp = distance(index_tip, index_mcp)
            middle_to_mcp = distance(middle_tip, middle_mcp)
            ring_to_mcp = distance(ring_tip, ring_mcp)
            pinky_to_mcp = distance(pinky_tip, pinky_mcp)
            
            # Open Hand (index finger extended for cursor)
            if thumb_index_dist > 0.06 and index_middle_dist > 0.06:
                is_open_hand = True
                debug_text = f"Index Finger: x={cursor_x}, y={cursor_y}, raw_x={raw_x:.3f}, {direction}"
            
            # Closed Fist
            elif index_to_mcp < 0.15 and middle_to_mcp < 0.15 and \
                 ring_to_mcp < 0.15 and pinky_to_mcp < 0.15:
                is_fist = True
                debug_text = f"Fist (Click), {direction}"
            
            # Scroll (Index and Middle Up)
            elif index_to_wrist > 0.08 and middle_to_wrist > 0.08 and \
                 ring_to_wrist < 0.2 and pinky_to_wrist < 0.2:
                is_scroll = True
                scroll_amount = -100 if index_tip.y < wrist.y else 100
                debug_text = f"Scroll {'Up' if scroll_amount < 0 else 'Down'}, {direction}"
            
            # Minimize: Pinky Up
            elif pinky_to_wrist > 0.08 and index_to_wrist < 0.2 and \
                 middle_to_wrist < 0.2 and ring_to_wrist < 0.2 and thumb_to_wrist < 0.2:
                is_minimize = True
                debug_text = f"Minimize (Pinky Up), {direction}"
            
            # Close: Thumbs Up
            elif thumb_to_wrist > 0.08 and index_to_wrist < 0.2 and \
                 middle_to_wrist < 0.2 and ring_to_wrist < 0.2 and pinky_to_wrist < 0.2:
                is_close = True
                debug_text = f"Thumbs Up (Close), {direction}"
            
            # OK Sign
            elif thumb_index_dist < 0.08 and middle_to_wrist > 0.08 and \
                 ring_to_wrist > 0.08 and pinky_to_wrist > 0.08:
                is_open_app = True
                debug_text = f"OK Sign (Double Click), {direction}"
            
            print(debug_text)
            return (cursor_x, cursor_y, is_fist, is_open_hand, is_scroll,
                    scroll_amount, is_minimize, is_close, is_open_app, debug_text, frame, raw_x)
    
    print(debug_text)
    return None, None, False, False, False, 0, False, False, False, debug_text, frame, None

def update_loop():
    global last_gesture_time, MIRROR_WEBCAM, smoothing_factor
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Error: Invalid webcam frame")
        return True, None  # Continue despite webcam error
    
    if MIRROR_WEBCAM:
        frame = cv2.flip(frame, 1)
    (cursor_x, cursor_y, is_fist, is_open_hand, is_scroll, scroll_amount,
     is_minimize, is_close, is_open_app, debug_text, frame_with_landmarks, raw_x) = detect_gestures(frame)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            print("Close button clicked, ignoring (use 'q' to quit)")
            continue  # Ignore window close button
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q:
                print("Quitting via 'q' key")
                return False, frame_with_landmarks
            elif event.key == pygame.K_m:
                MIRROR_WEBCAM = not MIRROR_WEBCAM
                print(f"Mirroring toggled to: {MIRROR_WEBCAM}")
            elif event.key == pygame.K_s:
                smoothing_factor = max(0.2, smoothing_factor - 0.05)
                print(f"Smoothing: {smoothing_factor:.2f}")
            elif event.key == pygame.K_S:
                smoothing_factor = min(0.6, smoothing_factor + 0.05)
                print(f"Smoothing: {smoothing_factor:.2f}")
    
    current_time = time.time()
    
    if cursor_x is not None and cursor_y is not None:
        print(f"Attempting to move cursor to x={cursor_x}, y={cursor_y}, raw_x={raw_x:.3f}")
        if is_open_hand:
            try:
                pyautogui.moveTo(cursor_x, cursor_y)
                print("Cursor moved successfully")
            except Exception as e:
                print(f"pyautogui error: {e}")
        elif current_time - last_gesture_time > gesture_debounce_delay:
            if is_fist:
                pyautogui.click()
                last_gesture_time = current_time
                print("Action: Left Click")
            elif is_scroll:
                pyautogui.scroll(scroll_amount)
                last_gesture_time = current_time
                print(f"Action: Scroll {'Up' if scroll_amount < 0 else 'Down'}")
            elif is_minimize:
                if platform.system().lower().startswith('win'):
                    pyautogui.hotkey('win', 'down')
                elif platform.system().lower() == 'darwin':
                    pyautogui.hotkey('command', 'm')
                else:
                    pyautogui.hotkey('alt', 'f9')
                last_gesture_time = current_time
                print("Action: Minimize Window")
            elif is_close:
                if platform.system().lower().startswith('win') or platform.system().lower() == 'linux':
                    pyautogui.hotkey('alt', 'f4')
                elif platform.system().lower() == 'darwin':
                    pyautogui.hotkey('command', 'w')
                last_gesture_time = current_time
                print("Action: Close Handled")
            elif is_open_app:
                pyautogui.doubleClick()
                last_gesture_time = current_time
                print("Action: Double Click")
    
    frame_rgb = cv2.cvtColor(frame_with_landmarks, cv2.COLOR_BGR2RGB)
    frame_rgb = np.rot90(frame_rgb)
    frame_surface = pygame.surfarray.make_surface(frame_rgb)
    frame_surface = pygame.transform.scale(frame_surface, (WEBCAM_WIDTH, WEBCAM_HEIGHT))
    
    screen.fill(WHITE)
    screen.blit(frame_surface, (WINDOW_WIDTH - WEBCAM_WIDTH, 0))
    
    if cursor_x is not None and cursor_y is not None:
        color = (BLUE if is_open_hand else
                 RED if is_fist else
                 GREEN if is_scroll and scroll_amount < 0 else
                 YELLOW if scroll_amount > 0 else
                 CYAN if is_minimize else
                 ORANGE if is_close else
                 MAGENTA if is_open_app else BLUE)
        pygame.draw.circle(screen, color, (cursor_x * WINDOW_WIDTH // SCREEN_WIDTH, cursor_y * WINDOW_HEIGHT // SCREEN_HEIGHT), 5)
    
    instructions = [
        "Index finger extended: Move cursor",
        "Fist: Left click",
        "Index+Middle up/down: Scroll",
        "Pinky up: Minimize",
        "Thumbs up: Close",
        "OK sign: Double-click",
        "Press 'q': Quit",
        "Press 'm': Toggle mirroring",
        "Press 's/S': Adjust smoothing"
    ]
    for i, text in enumerate(instructions):
        text_surface = font.render(text, True, (0, 0, 0))
        screen.blit(text_surface, (10, 10 + i * 30))
    
    debug_surface = font.render(debug_text, True, RED)
    screen.blit(debug_surface, (10, WINDOW_HEIGHT - 90))
    mirror_text = f"Mirroring: {'ON' if MIRROR_WEBCAM else 'OFF'}"
    mirror_surface = font.render(mirror_text, True, RED)
    screen.blit(mirror_surface, (10, WINDOW_HEIGHT - 60))
    smooth_text = f"Smoothing: {smoothing_factor:.2f}"
    smooth_surface = font.render(smooth_text, True, RED)
    screen.blit(smooth_surface, (10, WINDOW_HEIGHT - 30))
    
    pygame.display.flip()
    return True, frame_with_landmarks

async def main():
    clock = pygame.time.Clock()
    running = True
    while running:
        success, frame = update_loop()
        if not success:
            if frame is None:
                error_text = font.render("Webcam disconnected!", True, RED)
                screen.fill(WHITE)
                screen.blit(error_text, (WINDOW_WIDTH // 2 - 100, WINDOW_HEIGHT // 2))
                pygame.display.flip()
                await asyncio.sleep(1.0)
                continue
            running = False
        
        await asyncio.sleep(1.0 / 120)
    
    cap.release()
    pygame.quit()

if __name__ == "__main__":
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            asyncio.ensure_future(main())
            loop.run_forever()
        else:
            asyncio.run(main())
    except RuntimeError as e:
        print(f"Event loop error: {e}")